# AgentCore Memory Workshop: Building Strands Agents with Memory

Welcome to the AgentCore Memory Workshop! This notebook focuses on building intelligent agents with memory capabilities using Amazon Bedrock AgentCore Memory and Strands framework. You'll learn how to create agents that remember conversations and provide personalized experiences.

## Workshop Overview

### What You'll Learn

By the end of this workshop, you will be able to:

- **Understand** AgentCore Memory architecture and concepts
- **Create** memory resources for short-term conversation storage
- **Implement** memory hooks for automatic conversation management
- **Build** agents that remember user preferences and context
- **Test** memory functionality with practical examples
- **Apply** best practices for memory management in production

### Prerequisites

**⚠️ IMPORTANT: This workshop assumes you have completed the AgentCore Runtime Workshop first.**

This workshop reuses resources created in the Runtime Workshop to avoid duplication:
- ✅ **IAM Role**: `AgentCoreWorkshopRole` (will be updated with Memory permissions)
- ✅ **Cognito User Pool**: Authentication configuration
- ✅ **AgentCore Gateway**: Existing gateway with OpenAPI targets
- ✅ **API Credentials**: OpenWeather API key configuration

If you haven't completed the Runtime Workshop, some steps may fail. You should either:
1. **Recommended**: Complete the Runtime Workshop first
2. **Alternative**: Manually create the missing resources when prompted

### Additional Knowledge Requirements
- Strands framework fundamentals
- Building agents with tools and external APIs
- AWS services and IAM configuration

### AgentCore Memory

Amazon Bedrock AgentCore Memory is a managed service that provides:

- **Short-term memory** for conversation continuity
- **Long-term memory** with intelligent summarization
- **Automatic management** of memory lifecycle
- **Scalable storage** for multiple users and sessions
- **Integration** with Strands agents through hooks

## Setup Instructions

Let's verify your environment and install the required dependencies for AgentCore Memory.

### Step 1: Verify Python Version

In [ ]:
import sys
print(f"Python version: {sys.version}")

# Check if Python version is 3.10 or higher
if sys.version_info >= (3, 10):
    print("✅ Python version is compatible")
else:
    print("❌ Python 3.10+ is required. Please upgrade your Python version.")

### Step 2: Verify AWS Configuration

In [ ]:
import boto3
from botocore.exceptions import NoCredentialsError, ClientError

try:
    # Test AWS credentials
    session = boto3.Session()
    credentials = session.get_credentials()
    
    if credentials is None:
        print("❌ No AWS credentials found")
        print("Please configure AWS credentials using 'aws configure' or environment variables")
    else:
        print("✅ AWS credentials are configured")
        
        # Get current region
        region = session.region_name
        print(f"✅ AWS Region: {region}")
        
        # Test basic AWS access
        sts = boto3.client('sts')
        identity = sts.get_caller_identity()
        print(f"✅ AWS Account ID: {identity['Account']}")
        print(f"✅ AWS User/Role: {identity['Arn']}")
        
except NoCredentialsError:
    print("❌ AWS credentials not found")
    print("Please run 'aws configure' to set up your credentials")
except ClientError as e:
    print(f"❌ AWS access error: {e}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

### Step 3: Install AgentCore Memory Dependencies

Now let's install all the required Python packages for AgentCore Memory:

In [ ]:
!pip install strands-agents strands-agents-tools bedrock-agentcore boto3

### Step 4: Final Environment Validation

In [ ]:
import sys
import boto3
from datetime import datetime

print("🔍 AgentCore Memory Environment Validation")
print("=" * 45)

# System info
print(f"📅 Workshop Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🐍 Python Version: {sys.version.split()[0]}")
print(f"📍 Working Directory: {sys.path[0]}")

# AWS info
try:
    session = boto3.Session()
    region = session.region_name
    print(f"☁️  AWS Region: {region}")
    
    # Test Bedrock access
    bedrock = boto3.client('bedrock', region_name=region)
    print("✅ Bedrock service accessible")
    
    # Test AgentCore Memory access
    from bedrock_agentcore.memory import MemoryClient
    memory_client = MemoryClient(region_name=region)
    print("✅ AgentCore Memory client ready")
    
except Exception as e:
    print(f"⚠️  AWS setup issue: {e}")

print("\n🎉 Environment validation complete!")
print("\n📚 You're ready for AgentCore Memory development!")
print("\n👉 Proceed to the next section: 'Memory Basics'")

## Workshop Resource Management

This section handles the retrieval and setup of resources from the Runtime Workshop. The following resources will be reused:

- **IAM Role**: Retrieved from Runtime Workshop and updated with Memory permissions
- **Cognito Configuration**: Reused authentication setup
- **AgentCore Gateway**: Existing gateway with OpenAPI targets

### Environment Setup


In [1]:
# Set AWS credentials if not using Amazon SageMaker notebook
import os
# os.environ['AWS_ACCESS_KEY_ID'] = '' # Set the access key
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # Set the secret key
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')

In [2]:
import os
import sys

# Get the current directory (utils.py is in the same directory)
current_dir = os.getcwd()

# Add current directory to sys.path if not already there
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

# Now you can import utils
import utils

In [ ]:
#### Retrieve existing IAM role from Runtime Workshop
# Since we completed the Runtime Workshop first, we can reuse the existing IAM role
# instead of creating a new one. This avoids resource duplication.

import boto3
import importlib
import utils
importlib.reload(utils)

# Try to retrieve existing role first
iam_client = boto3.client('iam')
try:
    # Check if the role from Runtime Workshop exists
    role_response = iam_client.get_role(RoleName="AgentCoreWorkshopRole")
    agentcore_gateway_iam_role = {'Role': role_response['Role']}
    print("✅ Retrieved existing IAM role from Runtime Workshop")
    print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])
except iam_client.exceptions.NoSuchEntityException:
    # If role doesn't exist, create it (fallback for users who skipped Runtime Workshop)
    print("⚠️  Runtime Workshop role not found, creating new role...")
    agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("AgentCoreWorkshopRole")
    print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

### Retrieve existing Cognito User Pool from Runtime Workshop
Since the Runtime Workshop already created the Cognito resources, we can reuse them instead of creating duplicates.

In [ ]:


import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("✅ Retrieving existing Cognito resources from Runtime Workshop...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server confirmed.")

client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# Get discovery URL  
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(f"✅ Cognito Discovery URL: {cognito_discovery_url}")

In [ ]:
%store -r 
%store

In [ ]:
# Retrieve the Gateway from Runtime Workshop
# The Runtime Workshop created a Gateway that we can reuse for Memory functionality

import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])

# Try to find existing gateway by name first
gateway_name = 'TestGWforOpenAPI'
gateway_id = None
gateway_url = None

try:
    # List all gateways to find the one from Runtime Workshop
    list_response = gateway_client.list_gateways()
    existing_gateway = None
    
    for gateway in list_response.get('items', []):
        if gateway['name'] == gateway_name:
            existing_gateway = gateway
            break
    
    if existing_gateway:
        print(f"✅ Found existing Gateway from Runtime Workshop: {gateway_name}")
        gateway_id = existing_gateway['gatewayId']
        # Get full gateway details to retrieve gatewayUrl
        gateway_details = gateway_client.get_gateway(gatewayIdentifier=gateway_id)
        gateway_url = gateway_details['gatewayUrl']
        print(f"Gateway ID: {gateway_id}")
        print(f"Gateway URL: {gateway_url}")
    else:
        print("⚠️  No existing gateway found from Runtime Workshop")
        print("Please complete the Runtime Workshop first, or manually specify gateway ID below:")
        # Fallback: allow manual specification
        # gateway_id = "your-gateway-id-here"  # Uncomment and set if needed
        
except Exception as e:
    print(f"❌ Error retrieving gateway: {e}")
    print("Please ensure you have completed the Runtime Workshop first.")

# Memory Basics: Understanding AgentCore Memory

## What is AgentCore Memory?

AgentCore Memory provides two types of memory for your agents:

- 🧠 **Short-term Memory**: Raw conversation turns stored for immediate context
- 📚 **Long-term Memory**: Intelligent summaries and insights for persistent knowledge
- 🔄 **Automatic Management**: Handles memory lifecycle and optimization
- 🎯 **Session-based**: Organizes memory by actor and session identifiers

# Building the Weather Outfit Agent with Memory

Now let's build a more sophisticated agent that combines memory with external API calls. This agent will:

- 🌤️ **Remember user preferences** for weather and clothing
- 🌍 **Fetch real weather data** from OpenWeatherMap API
- 👔 **Provide personalized recommendations** based on memory
- 💾 **Store conversation context** for future interactions

### Step 1: Verify Docker Installation

In [ ]:
import subprocess
import sys

try:
    result = subprocess.run(['docker', '--version'], capture_output=True, text=True, check=True)
    print(f"✅ Docker is installed: {result.stdout.strip()}")
    
    # Check if Docker daemon is running
    result = subprocess.run(['docker', 'info'], capture_output=True, text=True, check=True)
    print("✅ Docker daemon is running")
    
except subprocess.CalledProcessError as e:
    print(f"❌ Docker issue: {e}")
    print("Please ensure Docker Desktop is installed and running.")
except FileNotFoundError:
    print("❌ Docker is not installed or not in PATH")
    print("Please install Docker Desktop from https://www.docker.com/products/docker-desktop")

### Step 2: Update IAM Role with Memory Permissions

**🔧 This is the ONLY modification needed from the Runtime Workshop resources.**

The existing `AgentCoreWorkshopRole` needs additional Memory permissions to support:
- Creating and managing memory resources
- Storing and retrieving conversation events
- Managing memory lifecycle

We'll update the inline policy with the complete set of permissions (including existing ones to ensure nothing breaks).

#### 1. Check for our current workshop IAM Role.

In [ ]:
# Search for current Workshop Role
iam_client = boto3.client('iam')
role_name = "AgentCoreWorkshopRole"
role_response = iam_client.get_role(RoleName=role_name)
agentcore_iam_role = role_response['Role']['Arn']
print(agentcore_iam_role)

#### 2. Update current IAM Role's inline policy
Let's update the inline policy to the role - "AgentCoreWorkshopRole".
(‼️ Remember to replace the "account-id" to your workshop's account id ‼️):

In [ ]:
%%writefile execution_role.json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "ECRImageAccess",
            "Effect": "Allow",
            "Action": [
                "ecr:GetDownloadUrlForLayer",
                "ecr:BatchGetImage",
                "ecr:GetAuthorizationToken"
            ],
            "Resource": [
                "arn:aws:ecr:us-east-1:<account-id>:repository/*"
            ]
        },
        {
            "Sid": "BedrockModelInvocation",
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "bedrock:InvokeModelWithResponseStream"
            ],
            "Resource": [
                "arn:aws:bedrock:us-east-1:<account-id>:*",
                "arn:aws:bedrock:*::foundation-model/*"
            ]
        },
        {
            "Sid": "BedrockAgentCoreAccess",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetWorkloadAccessToken",
                "bedrock-agentcore:GetEvent",
                "bedrock-agentcore:GetWorkloadAccessTokenForUserId",
                "bedrock-agentcore:GetWorkloadAccessTokenForJWT",
                "bedrock-agentcore:CreateEvent",
                "bedrock-agentcore:ListEvents",
                "bedrock-agentcore:CreateMemory",
                "bedrock-agentcore:DeleteMemory",
                "bedrock-agentcore:GetMemory",
                "bedrock-agentcore:ListMemories",
                "bedrock-agentcore:UpdateMemory",
                "bedrock-agentcore:ListMemoryEvents"
            ],
            "Resource": [
                "arn:aws:bedrock-agentcore:*:<account-id>:memory/*",
                "arn:aws:bedrock-agentcore:us-east-1:<account-id>:workload-identity-directory/default",
                "arn:aws:bedrock-agentcore:us-east-1:<account-id>:workload-identity-directory/default/workload-identity/agentcore_WS-*"
            ]
        },
        {
            "Sid": "CloudWatchLogsManagement",
            "Effect": "Allow",
            "Action": [
                "logs:DescribeLogStreams",
                "logs:CreateLogGroup"
            ],
            "Resource": [
                "arn:aws:logs:us-east-1:<account-id>:log-group:/aws/bedrock-agentcore/runtimes/*"
            ]
        },
        {
            "Sid": "CloudWatchMetrics",
            "Effect": "Allow",
            "Action": "cloudwatch:PutMetricData",
            "Resource": "*",
            "Condition": {
                "StringEquals": {
                    "cloudwatch:namespace": "bedrock-agentcore"
                }
            }
        },
        {
            "Sid": "ECRTokenAccess",
            "Effect": "Allow",
            "Action": [
                "ecr:GetAuthorizationToken"
            ],
            "Resource": "*"
        },
        {
            "Sid": "XRayTracing",
            "Effect": "Allow",
            "Action": [
                "xray:PutTelemetryRecords",
                "xray:GetSamplingRules",
                "xray:GetSamplingTargets",
                "xray:PutTraceSegments"
            ],
            "Resource": "*"
        },
        {
            "Sid": "CloudWatchLogsWrite",
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:us-east-1:<account-id>:log-group:/aws/bedrock-agentcore/runtimes/*:log-stream:*"
        },
        {
            "Sid": "CloudWatchLogsDescribe",
            "Effect": "Allow",
            "Action": "logs:DescribeLogGroups",
            "Resource": "arn:aws:logs:us-east-1:<account-id>:log-group:*"
        }
    ]
}

In [ ]:
## Update inline policy 

import boto3
import json

def update_inline_policy(role_name, policy_name, json_file_path):
    """
    Update inline policy for IAM role
    """
    iam = boto3.client('iam')
    
    # Read JSON policy file
    with open(json_file_path, 'r') as f:
        policy_document = json.load(f)
    
    try:
        # Put/update inline policy
        iam.put_role_policy(
            RoleName=role_name,
            PolicyName=policy_name,
            PolicyDocument=json.dumps(policy_document)
        )
        
        print(f"Inline policy {policy_name} updated for role {role_name}")
        return True
        
    except Exception as e:
        print(f"Error: {e}")
        return False

# Usage
role_name = "AgentCoreWorkshopRole"
json_file = "execution_role.json"
policy_name = "AgentCoreWorkshopPolicy"

update_inline_policy(role_name, policy_name, json_file)

### Step 3: Create a New ECR Repository
Next, let's create an ECR repository to store our agent's Docker image:

In [ ]:
# Create Workshop ECR Repository
import boto3
print("📦 Creating Workshop ECR Repository")
print("=" * 40)

def create_workshop_ecr():
    """Create ECR repository for workshop agents"""
    
    ecr_client = boto3.client('ecr')
    session = boto3.Session()
    region = session.region_name
    account_id = boto3.client('sts').get_caller_identity()['Account']
    repository_name = "agentcore-memory-workshop"
    
    try:
        # Create ECR repository
        print(f"Creating ECR repository: {repository_name}")
        response = ecr_client.create_repository(
            repositoryName=repository_name,
            imageScanningConfiguration={'scanOnPush': True},
            encryptionConfiguration={'encryptionType': 'AES256'}
        )
        
        repository_uri = response['repository']['repositoryUri']
        print(f"✅ ECR repository created successfully!")
        print(f"   Repository URI: {repository_uri}")
        
    except ecr_client.exceptions.RepositoryAlreadyExistsException:
        print(f"⚠️ Repository {repository_name} already exists")
        response = ecr_client.describe_repositories(repositoryNames=[repository_name])
        repository_uri = response['repositories'][0]['repositoryUri']
        print(f"   Using existing repository: {repository_uri}")
        
    except Exception as e:
        print(f"❌ Error creating ECR repository: {e}")
        return None
    
    return {
        'repository_uri': repository_uri,
        'registry_url': f"{account_id}.dkr.ecr.{region}.amazonaws.com",
        'region': region,
        'account_id': account_id
    }

# Create workshop ECR repository
workshop_ecr = create_workshop_ecr()

if workshop_ecr:
    workshop_ecr_url = workshop_ecr['repository_uri']
    print(f"\n🎉 Workshop ECR ready!")
    print(f"Repository URI: {workshop_ecr['repository_uri']}")
    print(f"Registry URL: {workshop_ecr['registry_url']}")
else:
    print("\n❌ Failed to create workshop ECR. Please check AWS permissions.")

### Step 4: Create your Memory Agent for AgentCore Runtime

Since we're going to deploy the basic agent with memory to agentcore runtime, we need to copy the Python code we previously wrote to the ```weather_agent_gateway_memory.py```.


In [ ]:
%%writefile weather_agent_gateway_memory.py

# Weather Agent with AgentCore Gateway and Memory Integration for Runtime
import os
import logging
import base64
import requests
from datetime import datetime
from strands import Agent
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent
from strands.models import BedrockModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.memory import MemoryClient
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient
from botocore.exceptions import ClientError

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("weather-agent-gateway-memory")

# Create AgentCore Runtime app
app = BedrockAgentCoreApp()

# Configuration - these will be set as environment variables or passed in payload
REGION = os.getenv('AWS_REGION', 'us-east-1')
GATEWAY_URL = os.getenv('GATEWAY_URL')
COGNITO_CLIENT_ID = os.getenv('COGNITO_CLIENT_ID')
COGNITO_CLIENT_SECRET = os.getenv('COGNITO_CLIENT_SECRET') 
COGNITO_USER_POOL_ID = os.getenv('COGNITO_USER_POOL_ID')
MEMORY_ID = os.getenv('MEMORY_ID')  # Set your memory ID here

logger.info(f"Configuration - Region: {REGION}")

class WeatherMemoryHook(HookProvider):
    """Enhanced memory hook for weather agent with better context management"""
    
    def __init__(self, memory_client: MemoryClient, memory_id: str, actor_id: str, session_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
        self.actor_id = actor_id
        self.session_id = session_id
        logger.info(f"Weather memory hook initialized for session: {session_id}")
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Load recent conversation history when agent starts"""
        try:
            logger.info(f"Loading weather conversation history for memory_id: {self.memory_id}")
            # Load the last 3 conversation turns from memory
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                k=3
            )
            
            if recent_turns:
                # Format conversation history for context
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role']
                        content = message['content']['text']
                        context_messages.append(f"{role}: {content}")
                
                context = "\n".join(context_messages)
                # Add context to agent's system prompt
                event.agent.system_prompt += f"\n\nRecent conversation history:\n{context}"
                logger.info(f"Loaded {len(recent_turns)} weather conversation turns")
            else:
                logger.info("No previous weather conversation history found")
                
        except Exception as e:
            logger.error(f"Weather memory load error: {e}")
    
    def on_message_added(self, event: MessageAddedEvent):
        """Store messages in memory with error handling"""
        messages = event.agent.messages
        latest_message = messages[-1]
        
        try:
            # Extract text content
            content_text = self.extract_message_content(latest_message)
            if content_text:
                logger.info(f"Saving message to memory_id: {self.memory_id}")
                self.memory_client.create_event(
                    memory_id=self.memory_id,
                    actor_id=self.actor_id,
                    session_id=self.session_id,
                    messages=[(content_text, latest_message["role"])]
                )
                logger.info(f"Successfully saved {latest_message['role']} message to weather memory")
        except Exception as e:
            logger.error(f"Weather memory save error: {e}")
    
    def extract_message_content(self, message):
        """Extract text content from message, handling different content types"""
        content = message.get("content", [])
        if not content:
            return None
        
        # Handle the first content item
        first_content = content[0]
        if isinstance(first_content, dict) and 'text' in first_content:
            return first_content['text']
        
        return None
    
    def register_hooks(self, registry: HookRegistry):
        # Register memory hooks
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

def get_cognito_token(user_pool_id, client_id, client_secret):
    """Get OAuth token from Cognito for Gateway access"""
    try:
        user_pool_domain = user_pool_id.lower().replace('_', '')
        token_endpoint = f"https://{user_pool_domain}.auth.{REGION}.amazoncognito.com/oauth2/token"
        
        logger.info(f"Using token endpoint: {token_endpoint}")
        
        # Prepare client credentials
        credentials = f"{client_id}:{client_secret}"
        encoded_credentials = base64.b64encode(credentials.encode()).decode()
        
        # Request token
        headers = {
            'Authorization': f'Basic {encoded_credentials}',
            'Content-Type': 'application/x-www-form-urlencoded'
        }
        
        data = {
            'grant_type': 'client_credentials',
            'scope': 'sample-agentcore-gateway-id/gateway:read sample-agentcore-gateway-id/gateway:write'
        }
        
        response = requests.post(token_endpoint, headers=headers, data=data)
        
        if response.status_code == 200:
            token_data = response.json()
            logger.info("Successfully obtained Cognito token")
            return token_data['access_token']
        else:
            logger.error(f"Failed to get token: {response.status_code} - {response.text}")
            return None
            
    except Exception as e:
        logger.error(f"Error getting Cognito token: {e}")
        return None

def create_mcp_client_transport(gateway_url, token):
    """Create MCP client transport with Gateway authentication"""
    return streamablehttp_client(gateway_url, headers={"Authorization": f"Bearer {token}"})

def ensure_memory_exists(memory_client, memory_name):
    """Ensure memory resource exists, create if it doesn't"""
    try:
        # Try to list memories and find existing one
        memories = memory_client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        
        if memory_id:
            logger.info(f"Using existing memory: {memory_id}")
            return memory_id
        else:
            logger.info(f"Creating new memory: {memory_name}")
            memory = memory_client.create_memory_and_wait(
                name=memory_name,
                strategies=[],  # No strategies for short-term memory
                description="Weather agent memory for gateway integration",
                event_expiry_days=7,
            )
            memory_id = memory['id']
            logger.info(f"Created new memory: {memory_id}")
            return memory_id
            
    except ClientError as e:
        if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
            # If memory already exists, retrieve its ID
            memories = memory_client.list_memories()
            memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
            logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
            return memory_id
        else:
            logger.error(f"Error managing memory: {e}")
            raise
    except Exception as e:
        logger.error(f"Unexpected error managing memory: {e}")
        raise

@app.entrypoint
def invoke_weather_agent_with_gateway_and_memory(payload):
    """Weather Agent with Gateway and Memory Integration"""
    
    # Get parameters from payload
    user_input = payload.get("prompt", "Hello")
    gateway_url = payload.get("gateway_url", GATEWAY_URL)
    memory_name = payload.get("memory_name", "WeatherAgentMemory")
    
    # Override config from payload if provided
    client_id = payload.get("cognito_client_id", COGNITO_CLIENT_ID)
    client_secret = payload.get("cognito_client_secret", COGNITO_CLIENT_SECRET)
    user_pool_id = payload.get("cognito_user_pool_id", COGNITO_USER_POOL_ID)
    
    # Actor and session IDs for memory
    actor_id = payload.get("actor_id", f"user-{datetime.now().strftime('%Y%m%d')}")
    session_id = payload.get("session_id", f"session-{datetime.now().strftime('%Y%m%d%H%M%S')}")
    
    logger.info(f"Processing request with Gateway URL: {gateway_url}")
    logger.info(f"User input: {user_input}")
    logger.info(f"Memory: {memory_name}, Actor: {actor_id}, Session: {session_id}")
    
    # Validate required parameters
    if not all([gateway_url, client_id, client_secret, user_pool_id]):
        missing = []
        if not gateway_url: missing.append("gateway_url")
        if not client_id: missing.append("cognito_client_id")
        if not client_secret: missing.append("cognito_client_secret") 
        if not user_pool_id: missing.append("cognito_user_pool_id")
        
        error_msg = f"Missing required parameters: {', '.join(missing)}"
        logger.error(error_msg)
        return {
            "error": error_msg,
            "missing_params": missing,
            "timestamp": datetime.now().isoformat()
        }
    
    try:
        # Initialize memory client
        memory_client = MemoryClient(region_name=REGION)
        
        # Ensure memory exists
        memory_id = ensure_memory_exists(memory_client, memory_name)
        
        # Get authentication token
        logger.info("Getting Cognito authentication token...")
        token = get_cognito_token(user_pool_id, client_id, client_secret)
        if not token:
            raise ValueError("Failed to get authentication token")
            
        logger.info("Successfully obtained authentication token")
        
        # Create MCP client for Gateway
        def create_transport():
            return create_mcp_client_transport(gateway_url, token)
        
        mcp_client = MCPClient(create_transport)
        
        # Setup Bedrock model
        model = BedrockModel(
            model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
            temperature=0.7
        )
        
        # Connect to Gateway and get tools
        with mcp_client:
            # Get available tools from Gateway
            tools = mcp_client.list_tools_sync()
            logger.info(f"Retrieved {len(tools) if tools else 0} tools from Gateway")
            
            if tools:
                tool_names = [tool.tool_name for tool in tools]
                logger.info(f"Available tools: {tool_names}")
            
            # Create memory hook
            memory_hook = WeatherMemoryHook(
                memory_client=memory_client,
                memory_id=memory_id,
                actor_id=actor_id,
                session_id=session_id
            )
            
            # Create enhanced system prompt for weather agent with memory
            system_prompt = """You are a professional weather assistant with both memory capabilities and access to OpenWeather API tools through an AgentCore Gateway.

            When users ask about weather:
            1. Use the available OpenWeather API tools to get current weather and forecasts
            2. Remember user preferences and previous conversations from your memory
            3. Provide clear, helpful weather information including temperature, conditions, humidity, and wind speed
            4. Give practical clothing and activity recommendations based on both weather conditions and remembered user preferences
            5. Reference previous conversations when relevant to provide personalized responses
            6. Be friendly and conversational in your responses
            7. If asked about multiple cities, provide useful comparisons
            8. Consider the user's context and provide relevant advice

            Available tools include:
            - getCurrentWeather: For current weather conditions in a specified location
            - getWeatherForecast: For 5-day weather forecasts

            Always use the tools to get real-time weather data rather than providing generic responses. When you cannot access weather data, explain what happened and suggest alternatives.
            
            Use your memory to provide personalized recommendations based on the user's past preferences and conversations."""
            
            # Create the agent with Gateway tools and memory
            agent = Agent(
                model=model,
                tools=tools if tools else [],
                hooks=[memory_hook],
                system_prompt=system_prompt
            )
            
            # Process user input
            logger.info(f"Processing user input with {len(tools) if tools else 0} tools and memory available")
            response = agent(user_input)
            
            # Extract response content safely
            try:
                if hasattr(response, 'message') and response.message:
                    if isinstance(response.message, dict) and 'content' in response.message:
                        content = response.message['content']
                        if isinstance(content, list) and len(content) > 0:
                            response_text = content[0].get('text', str(content))
                        else:
                            response_text = str(content)
                    else:
                        response_text = str(response.message)
                else:
                    response_text = str(response)
            except Exception as e:
                logger.warning(f"Error extracting response content: {e}")
                response_text = str(response)
            
            # Return structured response
            result = {
                "response": response_text,
                "timestamp": datetime.now().isoformat(),
                "tools_available": len(tools) if tools else 0,
                "memory_id": memory_id,
                "actor_id": actor_id,
                "session_id": session_id,
                "gateway_url": gateway_url,
                "status": "success"
            }
            
            logger.info("Successfully processed request with Gateway tools and memory")
            return result
            
    except Exception as e:
        logger.error(f"Error processing request: {str(e)}", exc_info=True)
        return {
            "error": str(e),
            "timestamp": datetime.now().isoformat(),
            "gateway_url": gateway_url,
            "status": "error",
            "troubleshooting": "Check Gateway URL, Cognito configuration, Memory setup, and ensure OpenWeather target is properly configured in Gateway"
        }

if __name__ == "__main__":
    app.run()

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
        
# Write environment configuration file
env_config = {
"GATEWAY_URL": gateway_url,
"COGNITO_CLIENT_ID": client_id,
"COGNITO_CLIENT_SECRET": client_secret,
"COGNITO_USER_POOL_ID": user_pool_id,
"AWS_REGION": REGION
}

with open('.env.runtime', 'w') as f:
    for key, value in env_config.items():
        f.write(f"{key}={value}\n")

print("📝 Created .env.runtime file with configuration")


#### Create a ```requirement.txt``` For Your AgentCore
Since we're going to create an docker image for the strands agent and deploy to AgentCore Runtime, we need a `requirement.txt` to include all the libraries.

In [ ]:
%%writefile requirements.txt
strands-agents
strands-agents-tools
bedrock-agentcore
boto3
mcp

### Step 5: Configure AgentCore Runtime

Now let's configure the runtime for deployment:

In [ ]:
# Configure basics agent for deployment
from bedrock_agentcore_starter_toolkit import Runtime
weather_agent_gateway_memory_runtime = Runtime()
weather_agent_gateway_memory_runtime_name = "weather_agent_gateway_memory"
region = "us-east-1"

weather_response = weather_agent_gateway_memory_runtime.configure(
    entrypoint="weather_agent_gateway_memory.py",
    execution_role=agentcore_iam_role,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    ecr_repository=workshop_ecr_url
)
weather_response

### Step 6: Launch Agent to AgentCore Runtime

Now we configured the agentcore runtime, let's launch the agent to AgentCore Runtime in AWS!
This will push the image to ECR repository and the AgentCore Runtime.

In [ ]:
launch_result = weather_agent_gateway_memory_runtime.launch()

### Step 7: Test your Agent with Short-Term Memory on AgentCore Runtime

In [ ]:
test_payload = {
    "prompt": "Hello, I'm Alice and I'm planning to visit Tokyo tomorrow. Can you check the weather for me?",
    "gateway_url": env_config["GATEWAY_URL"],
    "cognito_client_id": env_config["COGNITO_CLIENT_ID"],
    "cognito_client_secret": env_config["COGNITO_CLIENT_SECRET"],
    "cognito_user_pool_id": env_config["COGNITO_USER_POOL_ID"],
    "memory_name": "WeatherAgentGatewayMemory",
    "actor_id": "alice_test",
    "session_id": "test_session_001"
}
response = weather_agent_gateway_memory_runtime.invoke(test_payload)
print("✅ Test invocation successful!")
print(f"Response preview: {str(response)[:200]}...")

In [ ]:
from IPython.display import Markdown, display
import json

# 合併所有回應部分
full_response = b''.join(response['response']).decode("utf-8")

# 然後解析 JSON
response_data = json.loads(full_response)

# 顯示格式化的回應
print("=== Weather Agent Response ===")
display(Markdown(f"**Agent Response:**\n{response_data['response']}"))

print(f"\n**Status:** {response_data['status']}")
print(f"**Tools Available:** {response_data['tools_available']}")
print(f"**Timestamp:** {response_data['timestamp']}")

#### Test short-term memory persistence


In [ ]:

# Test memory persistence
print("\n🧠 Testing memory persistence...")
test_payload_2 = {
    "prompt": "Do you remember my name and where I wanted to visit?",
    "gateway_url": env_config["GATEWAY_URL"],
    "cognito_client_id": env_config["COGNITO_CLIENT_ID"],
    "cognito_client_secret": env_config["COGNITO_CLIENT_SECRET"],
    "cognito_user_pool_id": env_config["COGNITO_USER_POOL_ID"],
    "memory_name": "WeatherAgentGatewayMemory",
    "actor_id": "alice_test",
    "session_id": "test_session_001"  # Same session to test memory
}

response_2 = weather_agent_gateway_memory_runtime.invoke(test_payload_2)
print("✅ Memory persistence test successful!")
print(f"Memory response preview: {str(response_2)[:200]}...")

In [ ]:
from IPython.display import Markdown, display
import json

# 合併所有回應部分
full_response = b''.join(response_2['response']).decode("utf-8")

# 然後解析 JSON
response_data = json.loads(full_response)

# 顯示格式化的回應
print("=== Weather Agent Response ===")
display(Markdown(f"**Agent Response:**\n{response_data['response']}"))

print(f"\n**Status:** {response_data['status']}")
print(f"**Tools Available:** {response_data['tools_available']}")
print(f"**Timestamp:** {response_data['timestamp']}")

## Inspecting Memory Contents

Let's examine what's actually stored in our weather agent's memory:

In [ ]:
from bedrock_agentcore.memory import MemoryClient

print("🔍 Inspecting Weather Agent Memory Contents")
print("=" * 45)

# Initialize memory client
client = MemoryClient(region_name="us-east-1")

# Check what's stored in weather memory
recent_turns = client.get_last_k_turns(
    memory_id=memory_id,
    actor_id=actor_id,
    session_id=session_id,
    k=5 # Adjust k to see more or fewer turns
)

if recent_turns:
    print(f"Found {len(recent_turns)} conversation turns in memory:")
    print()
    
    for i, turn in enumerate(recent_turns, 1):
        print(f"📝 Turn {i}:")
        for message in turn:
            role = message['role']
            content = message['content']['text']
            # Truncate long messages for readability
            display_content = content[:150] + "..." if len(content) > 150 else content
            print(f"  {role}: {display_content}")
        print()
else:
    print("No conversation turns found in memory.")

print("✅ Memory inspection complete")

## Workshop Summary

Congratulations! You've successfully completed the AgentCore Memory Workshop. Here's what you've learned:

### 🎯 Key Concepts Covered

1. **Memory Basics**
   - Understanding short-term memory
   - Creating short-term memory resources
   - Basic short-term memory operations

2. **Memory Hooks**
   - Automatic conversation loading
   - Automatic message storage
   - Custom hook implementation

3. **Practical Implementation**
   - Building agents with short-term memory
   - Combining short-term memory with external APIs
   - Testing short-term memory

### 🛠️ What You Built

- ✅ Basic short-term memory-enabled agent
- ✅ Weather agent with API integration
- ✅ Custom short-term memory hooks for different scenarios
- ✅ Deploy your short-term memory agent on AgentCore Runtime

### 🚀 Next Steps

Now that you understand AgentCore Memory, you can:

1. **Explore Long-term Memory**: Add memory strategies for intelligent summarization
2. **Scale to Production**: Deploy your memory-enabled agents using AgentCore Runtime

### 📚 Additional Resources

- **AgentCore Memory** [AgentCore Memory Documentation](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/memory.html)
- **Strands Framework**: [Official Strands Documentation](https://strandsagents.com/latest/)
- **Amazon Bedrock**: [AWS Bedrock Documentation](https://aws.amazon.com/tw/bedrock/)

## Cleanup (Optional)

If you want to clean up the memory resources created during this workshop:

In [ ]:
print("🧹 Memory Cleanup (Optional)")
print("=" * 30)
from bedrock_agentcore.memory import MemoryClient

# Uncomment the lines below to delete memory resources
# WARNING: This will permanently delete all stored conversations
client = MemoryClient(region_name="<your-region>")
# Delete basic memory
memory_id = "<your-memory-id>"
client.delete_memory_and_wait(memory_id)
print(f"✅ Deleted basic memory: {memory_id}")

# Delete weather memory
# client.delete_memory_and_wait(weather_memory_id)
# print(f"✅ Deleted weather memory: {weather_memory_id}")

print("ℹ️ Cleanup code is commented out to prevent accidental deletion")
print("ℹ️ Uncomment the lines above if you want to delete the memory resources")